<div style="
    background-color:rgb(210, 230, 255);
    color:rgb(0, 0, 0);
    padding: 10px;
    border-radius: 10px;
    font-weight: bold;">
<h3>Modèle d'apprentissage automatique pour la prédiction d’usage le plus probable d'une friche en France</h3>
<ul>
    <li>Dans le cadre du projet PROFIL : https://github.com/heuzef/cartofriches</li>
    <li>Auteurs : Heuzef (https://heuzef.com) ; Frédéric Vincent (frederic.vincent@protonmail.com)</li>
    <li>Juin 2025</li>
    <li>Destiné au CEREMA</li>
    <li>Les données exploitées sont issues data.gouv.fr, respectant le Standard Friches (version 2022 - rev. v2023-12)</li>
</div>

# Import des librairies

In [1]:
!pip install -r requirements.txt

In [2]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import jinja2
import re

In [3]:
from sklearn import neighbors, svm, model_selection, preprocessing
from sklearn.model_selection import train_test_split

# Fonctions utiles

In [4]:
def audit(df):
    """
    Audite un DataFrame.
    
    Paramètres:
    df (pandas.DataFrame): Le DataFrame à analyser
    
    Retourne:
    pandas.DataFrame: Un DataFrame contenant pour chaque colonne:
                     - le nombre de valeurs manquantes
                     - le pourcentage de valeurs manquantes
                     - le type de données
                     - le nombre de valeurs uniques
    """
    # Calculer le nombre de valeurs manquantes par colonne
    missing_values = df.isnull().sum()
    
    # Calculer le pourcentage de valeurs manquantes
    missing_percentage = (missing_values / len(df)) * 100
    
    # Obtenir les types de données de chaque colonne
    dtypes = df.dtypes

    # Obtenir le nombre de valeurs uniques de chaque colonne
    nunique = df.nunique()
    
    # Créer un DataFrame avec les résultats
    audit = pd.DataFrame({
        'Manquantes': missing_values,
        '(%)': missing_percentage.round(2),
        'Type': dtypes,
        'Uniques': nunique
    })
    
    # Trier par pourcentage de valeurs manquantes (décroissant)
    audit = audit.sort_values('(%)', ascending=True)
    
    return audit

# Chargement des données

In [22]:
# Chargement des données silver

df_friches_silver = pd.read_csv("./data/friches_silver.csv", sep=",", low_memory=False)

print("Données des sites référencés dans Cartofriches disponible sur data.gouv.fr (respectant le standard CNIG) :")
print(f"{df_friches_silver.shape[0]} friches chargées — {df_friches_silver.shape[1]} variables")
print("Pas de doublon detectés")

display(df_friches_silver.head())

Données des sites référencés dans Cartofriches disponible sur data.gouv.fr (respectant le standard CNIG) :
2331 friches chargées — 54 variables
Pas de doublon detectés


,site_type,site_securite,site_reconv_type,bati_nombre,bati_pollution,bati_etat,proprio_personne,sol_pollution_existe,unite_fonciere_surface,longitude,...,urba_zone_type_n,urba_zone_type_u,urba_zone_type_zc,urba_zone_type_zca,urba_zone_type_znc,urba_doc_type_cc,urba_doc_type_plu,urba_doc_type_plui,urba_doc_type_psmv,urba_doc_type_u
0,friche d'habitat,0,0,2,0,0,personne morale,0,42877,1.337672,...,0,1,0,0,0,0,0,1,0,0
1,friche d'habitat,0,0,2,0,1,personne physique,0,143,-0.530774,...,0,1,0,0,0,0,1,0,0,0
2,friche agro-industrielle,0,1,0,0,0,personne physique,0,28389,-0.546245,...,1,0,0,0,0,0,1,0,0,0
3,friche d'habitat,0,2,3,0,1,personne physique,0,2619,-0.532152,...,0,1,0,0,0,0,1,0,0,0
4,friche agro-industrielle,0,1,0,0,0,personne physique,0,60709,-0.619462,...,0,0,0,0,0,0,1,0,0,0


# Extraction des données numériques

In [23]:
df_friches_silver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 54 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   site_type                                               1997 non-null   object 
 1   site_securite                                           2331 non-null   int64  
 2   site_reconv_type                                        2331 non-null   int64  
 3   bati_nombre                                             2331 non-null   int64  
 4   bati_pollution                                          2331 non-null   int64  
 5   bati_etat                                               2331 non-null   int64  
 6   proprio_personne                                        2162 non-null   object 
 7   sol_pollution_existe                                    2331 non-null   int64  
 8   unite_fonciere_surface                

In [24]:
df_friches_silver_numeriques = df_friches_silver.select_dtypes(exclude="object")

In [25]:
df_friches_silver_numeriques.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 52 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   site_securite                                           2331 non-null   int64  
 1   site_reconv_type                                        2331 non-null   int64  
 2   bati_nombre                                             2331 non-null   int64  
 3   bati_pollution                                          2331 non-null   int64  
 4   bati_etat                                               2331 non-null   int64  
 5   sol_pollution_existe                                    2331 non-null   int64  
 6   unite_fonciere_surface                                  2331 non-null   int64  
 7   longitude                                               2331 non-null   float64
 8   latitude                              

In [26]:
print(audit(df_friches_silver_numeriques))

                                                    Manquantes  (%)     Type  \
site_securite                                                0  0.0    int64   
site_reconv_type                                             0  0.0    int64   
bati_nombre                                                  0  0.0    int64   
bati_pollution                                               0  0.0    int64   
bati_etat                                                    0  0.0    int64   
sol_pollution_existe                                         0  0.0    int64   
unite_fonciere_surface                                       0  0.0    int64   
longitude                                                    0  0.0  float64   
latitude                                                     0  0.0  float64   
uzf_activite_agricole                                        0  0.0    int64   
uzf_activite_economique                                      0  0.0    int64   
uzf_autres                              

# Création des jeux d'apprentissage et de test. 

In [27]:
X = df_friches_silver_numeriques.drop(columns=['site_reconv_type'])

In [28]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 51 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   site_securite                                           2331 non-null   int64  
 1   bati_nombre                                             2331 non-null   int64  
 2   bati_pollution                                          2331 non-null   int64  
 3   bati_etat                                               2331 non-null   int64  
 4   sol_pollution_existe                                    2331 non-null   int64  
 5   unite_fonciere_surface                                  2331 non-null   int64  
 6   longitude                                               2331 non-null   float64
 7   latitude                                                2331 non-null   float64
 8   uzf_activite_agricole                 

In [29]:
Y = df_friches_silver_numeriques['site_reconv_type']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

# Classification avec un modèle KNN

In [31]:
knn = neighbors.KNeighborsClassifier(n_neighbors=7, metric='minkowski')
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [32]:
y_pred = knn.predict(X_test)
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,0,1,2,3,4,6,10
Classe réelle,,,,,,,
0,181,0,6,13,2,2,0
1,18,1,0,2,2,0,0
2,37,0,0,1,2,3,0
3,53,0,0,8,4,0,1
4,51,0,1,9,7,0,0
5,11,0,0,1,0,0,0
6,15,0,0,5,1,0,0
7,4,0,0,3,1,0,0
9,13,0,0,0,3,0,0


In [33]:
knn.score(X_test, y_test)

0.42184154175588867

# Classification avec un modèle SVM

In [34]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [35]:
clf = svm.SVC(gamma=0.01, kernel='poly')

In [36]:
clf.fit(X_train_scaled, y_train)

SVC(gamma=0.01, kernel='poly')

In [37]:
y_pred_svm = clf.predict(X_test_scaled)
pd.crosstab(y_test, y_pred_svm, rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,0,1,2,3,4,5,6
Classe réelle,,,,,,,
0,197,0,0,2,2,1,2
1,21,1,0,0,1,0,0
2,42,0,0,0,0,0,1
3,64,0,0,1,0,0,1
4,62,0,1,0,5,0,0
5,11,0,0,0,0,0,1
6,15,1,0,0,2,0,3
7,8,0,0,0,0,0,0
9,15,0,0,0,0,0,1


In [38]:
clf.score(X_test_scaled, y_test)

0.44325481798715205

# Classification avec un modèle SVM et une grille de recherche

In [39]:
parametres = {'C':[0.1, 1, 10], 'kernel':['rbf', 'linear', 'poly'], 'gamma':[0.001, 0.1, 0.5]}

In [40]:
grid_clf = model_selection.GridSearchCV(estimator=clf, param_grid=parametres)